# TODO

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1AoolDYePUpPkRCKIu0cP9zV7lX5QGD3Z?usp=sharing)

In [1]:
%cd ..

/home/ax37/code/py/rudallel


In [2]:
# !conda install -c anaconda cudatoolkit=10.2

In [3]:
# !pip install torch==1.10.0

In [4]:
!pip uninstall -y rudalle
!pip install -e .  # rudalle==0.0.1rc8 > /dev/null

Found existing installation: rudalle 1.1.0rc0
Uninstalling rudalle-1.1.0rc0:
  Successfully uninstalled rudalle-1.1.0rc0
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Obtaining file:///home/ax37/code/py/rudallel
  Preparing metadata (setup.py) ... done


  Running setup.py develop for rudalle
You should consider upgrading via the '/home/ax37/anaconda3/bin/python -m pip install --upgrade pip' command.


In [5]:
#@param {type: "float"}
ALLOWED_MEMORY = 6.5 # 3.5 # choose your GPU memory in GB  
if ALLOWED_MEMORY < 4.5:
    DALLE_BS = 1
elif ALLOWED_MEMORY < 5.5:
    DALLE_BS = 2
elif ALLOWED_MEMORY < 6.5:
    DALLE_BS = 3
elif ALLOWED_MEMORY < 7.5:
    DALLE_BS = 4
elif ALLOWED_MEMORY < 8.5:
    DALLE_BS = 5
elif ALLOWED_MEMORY < 9.5:
    DALLE_BS = 6
elif ALLOWED_MEMORY < 10.5:
    DALLE_BS = 7
else:
    DALLE_BS = 8
    
print('ruDALL-E batch size:', DALLE_BS)

ruDALL-E batch size: 4


In [6]:
import multiprocessing
import torch
from psutil import virtual_memory


ram_gb = round(virtual_memory().total / 1024**3, 1)
print('CPU:', multiprocessing.cpu_count())
print('RAM GB:', ram_gb)
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())

!nvidia-smi

CPU: 16
RAM GB: 31.4
PyTorch version: 1.10.0+cu102
CUDA version: 10.2
cuDNN version: 7605
/bin/sh: line 1: nvidia-smi: command not found


In [7]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [8]:
import transformers
import more_itertools
from tqdm.auto import tqdm

from rudalle.pipelines import generate_images, show  # , cherry_pick_by_clip
from rudalle import get_rudalle_model, get_tokenizer, get_vae  # , get_ruclip
from rudalle.utils import seed_everything, torch_tensors_to_pil_list

In [9]:
# # prepare models:
device = "cuda"
dalle = get_rudalle_model("Malevich", pretrained=True, fp16=True, device=device)
tokenizer = get_tokenizer()
vae = get_vae(dwt=True)

# # prepare utils:
# ruclip, ruclip_processor = get_ruclip("ruclip-vit-base-patch32-v5")

◼️ Malevich is 1.3 billion params model from the family GPT3-like, that uses Russian language and text+image multi-modality.
tokenizer --> ready
Working with z of shape (1, 256, 32, 32) = 262144 dimensions.
vae --> ready


In [10]:
from rudalle.dalle.transformer import ELAttention

In [11]:
# import torch
# import gc


# gc.collect()
# torch.cuda.empty_cache()
# torch.set_grad_enabled(False)
# att1 = dalle.module.transformer.layers[0].attention
# att1.custom_relax = False
# att1.cogview_pb_relax = False
# att1.eval()
# att1.attention_dropout = torch.nn.Identity()
# att2 = ELAttention(att1)
# torch.manual_seed(42)
# inputs = (torch.randn((2, 4, dalle.module.transformer.layers[0].attention.hidden_size)).cuda().half(),
#           torch.ones((1, 1, 4, 4)).cuda().half())
# x1, _ = att1(*inputs, use_cache=True)
# x2, _ = att2(*inputs, use_cache=True)
# diff = x1 - x2
# print((diff ** 2).sum())

# diff = diff.clamp(-4, 4)

# from matplotlib import pyplot as plt
# plt.imshow(diff.reshape(-1, diff.shape[-1] // 16).detach().float().cpu().numpy() ** 2)
# plt.colorbar()

In [12]:
vae.cuda()
0

0

In [13]:
with torch.cuda.amp.autocast():
    dalle.module.transformer.to_el()
0

RuntimeError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 7.79 GiB total capacity; 4.33 GiB already allocated; 146.44 MiB free; 4.38 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
text = "эль-внимание"

seed_everything(14)
images = []
for top_k, top_p, images_num in [
    (2048, 0.995, 8),
#     (1536, 0.99, 8),
#     (1024, 0.99, 8),
#     (1024, 0.98, 8),
#     (512, 0.97, 8),
#     (384, 0.96, 8),
#     (256, 0.95, 8),
#     (128, 0.95, 8),
]:
    images += generate_images(text, tokenizer, dalle, vae,
                                 top_k=top_k, images_num=images_num,
                                 top_p=top_p, bs=DALLE_BS,
                                 use_cache=False)
show(images[0])

In [ ]:
show(images[0])

In [ ]:
pil_images = []
for _codebooks in tqdm(torch.cat(codebooks).cpu()):
    with torch.no_grad():
        images = vae.decode(_codebooks.unsqueeze(0))
        pil_images += torch_tensors_to_pil_list(images)